In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('insurance_pre.csv')
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset    

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [4]:
indep=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
dep=dataset[['charges']]

In [5]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
#from sklearn.tree import DecisionTreeRegressor
param_grid = {'criterion':['mse','mae'],
              'max_features': ['auto','sqrt','log2'],
              'n_estimators':[10,100]}
grid = GridSearchCV(RandomForestRegressor(), param_grid, refit = True, verbose= 3,n_jobs=-1)
# fitting the model for grid search
grid.fit(X_train, y_train)

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  26 out of  36 | elapsed:    3.3s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    4.0s finished
C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'criterion': ['mse', 'mae'],

In [8]:
# print best parameter after tuning
#print(grid.best_params_)
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test)
# print classification report
from sklearn.metrics import r2_score
r_score=r2_score(y_test,grid_predictions)
print("The R_score value for best parameter {}:".format(grid.best_params_),r_score)

The R_score value for best parameter {'criterion': 'mse', 'max_features': 'log2', 'n_estimators': 100}: 0.8722821294720762


In [9]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.035220,0.013685,0.000759,0.001073,mse,auto,10,"{'criterion': 'mse', 'max_features': 'auto', '...",0.776353,0.777965,0.793361,0.782553,0.007664,9
1,0.185374,0.014848,0.014918,0.000996,mse,auto,100,"{'criterion': 'mse', 'max_features': 'auto', '...",0.785555,0.791047,0.795240,0.790609,0.003967,5
2,0.020967,0.007272,0.005208,0.007365,mse,sqrt,10,"{'criterion': 'mse', 'max_features': 'sqrt', '...",0.779853,0.769315,0.789588,0.779586,0.008274,11
3,0.168504,0.006270,0.012761,0.004064,mse,sqrt,100,"{'criterion': 'mse', 'max_features': 'sqrt', '...",0.805548,0.807254,0.793029,0.801948,0.006340,2
4,0.027648,0.005378,0.000000,0.000000,mse,log2,10,"{'criterion': 'mse', 'max_features': 'log2', '...",0.794188,0.795352,0.773306,0.787623,0.010126,7
5,0.141080,0.000199,0.015638,0.000009,mse,log2,100,"{'criterion': 'mse', 'max_features': 'log2', '...",0.808638,0.801197,0.796789,0.802215,0.004892,1
6,0.110912,0.012354,0.005567,0.007139,mae,auto,10,"{'criterion': 'mae', 'max_features': 'auto', '...",0.763233,0.777329,0.787869,0.776129,0.010096,12
7,0.916929,0.020226,0.005715,0.007043,mae,auto,100,"{'criterion': 'mae', 'max_features': 'auto', '...",0.789374,0.785552,0.791324,0.788751,0.002396,6
8,0.079779,0.014829,0.000000,0.000000,mae,sqrt,10,"{'criterion': 'mae', 'max_features': 'sqrt', '...",0.780029,0.777974,0.787407,0.781801,0.004049,10
9,0.551752,0.012758,0.010419,0.007368,mae,sqrt,100,"{'criterion': 'mae', 'max_features': 'sqrt', '...",0.804473,0.801899,0.794520,0.800302,0.004218,4


In [10]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age:4
BMI:56
Children:4
Sex Male 0 or 1:1
Smoker Yes 0 or 1:0


In [11]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])#␣change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[15595.5789975]
